In [1]:
#PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=cpp
import pandas as pd
import numpy as np
import pickle
import os
import joblib

from keras.models import Sequential
from tensorflow.keras.regularizers import L1, L2
from keras.layers import Dropout
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
#from Funciones_tablas_validacion import *

from sklearn.model_selection import GridSearchCV

#Funciones que me paso Emmanuel
from sklearn.feature_selection import SelectFpr, SelectKBest, f_classif, chi2 
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

import sys
sys.path.append('C:/Users/RobertoRosasGuerrero/Analítica Luisito Valdez/SCRIPTS/SCRIPTS GENERALES PYTHON')
from Funciones_tablas_validacion import *


import warnings
warnings.filterwarnings("ignore")

#### Aplicar features selection a la base, smothe y partición de entrenamiento y Test

In [2]:
#   SUBIMOS LA BASE CON WOE PARA APLICAR UN FEATURE SELECTION Y UN SMOTE
df_woe = pd.read_csv("ActivoIMSS_AceptadosyRechazados_S3_CN_WOE.csv")
#df_woe.rename(columns={'BGI.1.':'BGI'}, inplace = True)
df_woe.drop(df_woe.filter(regex='^Unnamed:',axis=1),1,inplace=True)
#Si no es aceptado ponemos 0, pero si lo aceptamos le ponemos un 1
df_woe["aceptado"]=np.where(df_woe["aceptado"]=="no",0,1)
df_woe.head()

,idconsulta,BGI,IND_2,aceptado,$R-BGI,$RC-BGI,$RI-BGI,AYC_008_Mean_woe,AYC_002_Max_woe,AYC_019_Max_woe,...,tipoContrato_5_INT_135_MOR_Max_woe,tipoContrato_5_INT_136_MOR_Max_woe,tipoContrato_2_INT_137_MOR_Max_woe,tipoContrato_5_INT_137_MOR_Max_woe,tipoContrato_2_INT_138_MOR_Max_woe,tipoContrato_1_INT_142_MOR_Max_woe,tipoContrato_1_INT_147_MOR_Max_woe,tipoContrato_1_INT_148_MOR_Max_woe,tipoContrato_5_CON_001_BUR_Sum_woe,tipoContrato_5_CON_002_BUR_Sum_woe
0,942380,0,2021_2,0,0,0.773585,66,0.219752,-0.033989,-0.066562,...,0.178034,0.171732,-0.241524,0.174589,-0.240214,0.096674,0.087699,0.086188,-0.000989,-0.000989
1,857713,0,2020_11,0,0,0.773585,66,-0.573297,-0.033989,0.257140,...,-0.108060,-0.098366,0.100509,-0.095593,0.102867,0.051979,-0.633934,-0.464178,-0.523191,-0.745429
2,911339,1,2021_1,0,0,0.773585,66,0.219752,-0.033989,-0.066562,...,0.178034,0.171732,0.100509,0.174589,0.102867,0.096674,0.087699,0.086188,-0.000989,-0.000989
3,801699,1,2020_9,0,0,0.773585,66,0.219752,-0.033989,-0.066562,...,0.178034,0.171732,0.066324,0.174589,0.066324,-0.819202,-0.633934,-0.464178,-0.523191,-0.745429
4,903992,0,2021_1,0,0,0.773585,66,0.219752,-0.033989,-0.066562,...,-0.160496,-0.160496,0.066324,-0.160496,0.066324,0.096674,0.087699,0.086188,-0.000989,-0.000989


In [ ]:
# first_column = df_woe.pop('idcredit') 
# df_woe.insert(0, 'idcredit', first_column) 

In [3]:
df_woe.shape

(1376, 2202)

In [4]:
df_woe.groupby("BGI")["BGI"].count()

BGI
0    1065
1     311
Name: BGI, dtype: int64

In [5]:
p_train = 0.70 # Porcentaje de train.

df_woe['is_train'] = np.random.uniform(0, 1, len(df_woe)) <= p_train
#train, train = df_woe[df_woe['is_train']==True], df_woe[df_woe['is_train']==False]
#df_woe = df_woe.drop('is_train', 1)


In [13]:
df_woe.head(2)

,idconsulta,is_train,IND_2,aceptado,BGI,AYC_008_Mean_woe,AYC_002_Max_woe,AYC_019_Max_woe,AYC_021_Max_woe,AYC_022_Max_woe,...,tipoContrato_5_INT_135_MOR_Max_woe,tipoContrato_5_INT_136_MOR_Max_woe,tipoContrato_2_INT_137_MOR_Max_woe,tipoContrato_5_INT_137_MOR_Max_woe,tipoContrato_2_INT_138_MOR_Max_woe,tipoContrato_1_INT_142_MOR_Max_woe,tipoContrato_1_INT_147_MOR_Max_woe,tipoContrato_1_INT_148_MOR_Max_woe,tipoContrato_5_CON_001_BUR_Sum_woe,tipoContrato_5_CON_002_BUR_Sum_woe
0,942380,False,2021_2,0,0,0.219752,-0.033989,-0.066562,0.140651,0.109964,...,0.178034,0.171732,-0.241524,0.174589,-0.240214,0.096674,0.087699,0.086188,-0.000989,-0.000989
1,857713,True,2020_11,0,0,-0.573297,-0.033989,0.257140,0.140651,0.109964,...,-0.108060,-0.098366,0.100509,-0.095593,0.102867,0.051979,-0.633934,-0.464178,-0.523191,-0.745429


In [14]:
df_woe.drop(["$R-BGI","$RC-BGI","$RI-BGI"],1,inplace=True)

KeyError: "['$R-BGI', '$RC-BGI', '$RI-BGI'] not found in axis"

In [ ]:
first_column = df_woe.pop('is_train') 
df_woe.insert(2, 'is_train', first_column) 

first_column = df_woe.pop('BGI') 
df_woe.insert(4, 'BGI', first_column) 

In [ ]:
df_woe.iloc[:,5:].head(2)

,AYC_008_Mean_woe,AYC_002_Max_woe,AYC_019_Max_woe,AYC_021_Max_woe,AYC_022_Max_woe,AYC_023_Max_woe,nombreOtorgante_13_AYC_005_Sum_woe,nombreOtorgante_12_AYC_008_Sum_woe,nombreOtorgante_12_AYC_009_Sum_woe,nombreOtorgante_12_AYC_010_Sum_woe,...,tipoContrato_5_INT_135_MOR_Max_woe,tipoContrato_5_INT_136_MOR_Max_woe,tipoContrato_2_INT_137_MOR_Max_woe,tipoContrato_5_INT_137_MOR_Max_woe,tipoContrato_2_INT_138_MOR_Max_woe,tipoContrato_1_INT_142_MOR_Max_woe,tipoContrato_1_INT_147_MOR_Max_woe,tipoContrato_1_INT_148_MOR_Max_woe,tipoContrato_5_CON_001_BUR_Sum_woe,tipoContrato_5_CON_002_BUR_Sum_woe
0,0.219752,-0.033989,-0.066562,0.140651,0.109964,0.082735,-0.032695,-0.048943,-0.060107,-0.049126,...,0.178034,0.171732,-0.241524,0.174589,-0.240214,0.096674,0.087699,0.086188,-0.000989,-0.000989
1,-0.573297,-0.033989,0.257140,0.140651,0.109964,0.082735,0.261804,-0.048943,-0.060107,-0.049126,...,-0.108060,-0.098366,0.100509,-0.095593,0.102867,0.051979,-0.633934,-0.464178,-0.523191,-0.745429


In [15]:
X_woe = df_woe.iloc[:,5:]
y_woe = df_woe['BGI']

# define the selector to obtain the best 35 features
selector = SelectKBest(f_classif, k=35)
selector.fit(X_woe,y_woe)
new_features = selector.get_feature_names_out()

In [16]:
#CHECAR SI YA GENERAMOS LA DATA O SI ES LA PRIMERA VEZ QUE LA VAMOS A GENERAR
df_1 = df_woe[["idconsulta","IND_2","is_train","aceptado","BGI"]+ list(new_features)] #,'RFC.10' ,'fecha_consulta','vOpenned'  ,
#df_1 = df_woe[["idconsulta","vOpenned","iCredId","vAfiliado","vBanco","BGI","is_train"]+ list(new_features)]

#df_1 = pd.read_csv("C:/Users/RobertoRosasGuerrero/Analítica Luisito Valdez/IBILLETE/Primer Modelo Ibillete 14-06-2022/ROBERTO/ROBERTO_BASE_MODELAR_S2.csv")


In [17]:
print(df_1.shape)
display(df_1.head(1))

(1376, 40)


,idconsulta,IND_2,is_train,aceptado,BGI,DOM_009_BUR_Max_woe,nombreOtorgante_1_EXP_003_Sum_woe,NUM_002_BUR_Mean_woe,RES_033_BUR_woe,INT_050_MOR_Mean_woe,...,nombreOtorgante_8_INT_052_MOR_Max_woe,nombreOtorgante_8_INT_053_MOR_Max_woe,nombreOtorgante_8_INT_064_MOR_Max_woe,nombreOtorgante_8_INT_065_MOR_Max_woe,nombreOtorgante_8_INT_071_MOR_Max_woe,nombreOtorgante_8_INT_072_MOR_Max_woe,nombreOtorgante_8_INT_076_MOR_Max_woe,nombreOtorgante_8_INT_077_MOR_Max_woe,nombreOtorgante_8_INT_083_MOR_Max_woe,nombreOtorgante_8_INT_089_MOR_Max_woe
0,942380,2021_2,False,0,0,-0.384485,0.06379,-0.328648,0.033898,-0.358076,...,0.126249,0.127186,0.170703,0.170703,0.170703,0.170703,0.170703,0.170703,0.170703,0.170703


In [18]:
# # define the new features
# X_new = df_woe[new_features]
# y_new = y_woe

# #apply SMOTE algoritm
# X_res, y_res = SMOTE(random_state=42).fit_resample(X_new, y_new)

# define the new features
X_new = df_woe[['idconsulta','aceptado',"is_train","IND_2"]+ list(new_features)]
y_new = y_woe

#apply SMOTE algoritm
X_res, y_res = SMOTE(random_state=42).fit_resample(X_new, y_new)

In [19]:
# print(df_woe.groupby("BGI")["BGI"].count())
# print(y_woe.sum())
# print(y_woe.count())
# print(y_res.sum())
# print(y_res.count())
print(X_new.shape)
print(y_new.shape)
print(X_res.shape)
print(y_res.shape)


(1376, 39)
(1376,)
(2130, 39)
(2130,)


In [ ]:
#df_1 = df_woe[['idconsulta', 'id.credit',"BGI"] + list(new_features)] #take only selected features and target


In [20]:
#print(df_1["BGI"].sum())
#print(df_1["BGI"].count())
print(y_res.sum())
print(y_res.count())

1065
2130


In [ ]:
## BASE CON SMOTE
# df = pd.read_csv("BASE_MODELAR_S2_S.csv",encoding ='latin-1')
# df.rename(columns={'BGI.1.':'BGI'}, inplace = True)
# df.head()

In [21]:
df = pd.concat([X_res,y_res],axis=1)
#df= pd.read_csv("C:/Users/RobertoRosasGuerrero/Analítica Luisito Valdez/IBILLETE/Primer Modelo Ibillete 14-06-2022/ROBERTO/ROBERTO_BASE_MODELAR_S2_S.csv")

In [22]:
df.shape

(2130, 40)

In [23]:
df.head(2)


,idconsulta,aceptado,is_train,IND_2,DOM_009_BUR_Max_woe,nombreOtorgante_1_EXP_003_Sum_woe,NUM_002_BUR_Mean_woe,RES_033_BUR_woe,INT_050_MOR_Mean_woe,nombreOtorgante_8_INT_061_MOR_Sum_woe,...,nombreOtorgante_8_INT_053_MOR_Max_woe,nombreOtorgante_8_INT_064_MOR_Max_woe,nombreOtorgante_8_INT_065_MOR_Max_woe,nombreOtorgante_8_INT_071_MOR_Max_woe,nombreOtorgante_8_INT_072_MOR_Max_woe,nombreOtorgante_8_INT_076_MOR_Max_woe,nombreOtorgante_8_INT_077_MOR_Max_woe,nombreOtorgante_8_INT_083_MOR_Max_woe,nombreOtorgante_8_INT_089_MOR_Max_woe,BGI
0,942380,0,False,20212.0,-0.384485,0.06379,-0.328648,0.033898,-0.358076,0.170703,...,0.127186,0.170703,0.170703,0.170703,0.170703,0.170703,0.170703,0.170703,0.170703,0
1,857713,0,True,202011.0,0.283191,-0.78465,-0.328648,-0.937950,0.110101,-0.314646,...,-0.478071,-0.694359,-0.358449,-0.451612,-0.389045,-0.411038,-0.460498,-0.439350,-0.420007,0


In [24]:
df_1.to_csv("BASE_MODELAR_IMSS_ACT_Acep_Rech_S3.csv",index=False)
df.to_csv("BASE_MODELAR_IMSS_ACT_Acep_Rech_S3_S.csv")

### Hasta aquí el proceso de features selection, Smote, segmentación de entrenamiento y test

#### Si ya estaban las bases solo las cargamos 

In [ ]:
# BASE CON PARTICION
df_1 = pd.read_csv("BASE_MODELAR_IMSS_ACT_Acep_Rech_S2.csv",encoding = 'latin-1')
df_1.drop(df_1.filter(regex='^Unnamed:',axis=1),1,inplace=True)
#df_1.rename(columns={'BGI.1.':'BGI'}, inplace = True)
df_1.head()
#df_1.groupby("BGI")["BGI"].count()

In [ ]:
# BASE CON SMOTE
df = pd.read_csv("BASE_MODELAR_IMSS_ACT_Acep_Rech_S2_S.csv",encoding = 'latin-1')
df.drop(df.filter(regex='^Unnamed:',axis=1),1,inplace=True)
#df.rename(columns={'BGI.1.':'BGI'}, inplace = True)
df.head()

### Observaciones 
#### df_1 es la base original
#### X_new, y_new son las bases originales
#### X_res, y_res son las bases que le aplicamos Smote
#### df es la base completa con Smote

In [25]:
### TRAIN Y TEST GENERADOS CON BASE SMOTE
df_train = df['is_train'] == True
df_train = df[df_train]
#df_train = df['Partición'] == "1_Entrenamiento"
#df_train = df[df_train]

In [26]:
df_train.head(2)

,idconsulta,aceptado,is_train,IND_2,DOM_009_BUR_Max_woe,nombreOtorgante_1_EXP_003_Sum_woe,NUM_002_BUR_Mean_woe,RES_033_BUR_woe,INT_050_MOR_Mean_woe,nombreOtorgante_8_INT_061_MOR_Sum_woe,...,nombreOtorgante_8_INT_053_MOR_Max_woe,nombreOtorgante_8_INT_064_MOR_Max_woe,nombreOtorgante_8_INT_065_MOR_Max_woe,nombreOtorgante_8_INT_071_MOR_Max_woe,nombreOtorgante_8_INT_072_MOR_Max_woe,nombreOtorgante_8_INT_076_MOR_Max_woe,nombreOtorgante_8_INT_077_MOR_Max_woe,nombreOtorgante_8_INT_083_MOR_Max_woe,nombreOtorgante_8_INT_089_MOR_Max_woe,BGI
1,857713,0,True,202011.0,0.283191,-0.78465,-0.328648,-0.93795,0.110101,-0.314646,...,-0.478071,-0.694359,-0.358449,-0.451612,-0.389045,-0.411038,-0.460498,-0.43935,-0.420007,0
2,911339,0,True,20211.0,-0.384485,0.06379,0.206650,-0.93795,-0.358076,0.179455,...,0.237728,0.223721,0.179650,0.192475,0.197558,0.202075,0.186129,0.18005,0.186383,1


In [27]:
df_train.iloc[:,4:-1].head(2)

,DOM_009_BUR_Max_woe,nombreOtorgante_1_EXP_003_Sum_woe,NUM_002_BUR_Mean_woe,RES_033_BUR_woe,INT_050_MOR_Mean_woe,nombreOtorgante_8_INT_061_MOR_Sum_woe,nombreOtorgante_8_INT_064_MOR_Sum_woe,nombreOtorgante_8_INT_066_MOR_Sum_woe,nombreOtorgante_8_INT_061_MOR_Mean_woe,nombreOtorgante_8_INT_062_MOR_Mean_woe,...,nombreOtorgante_8_INT_052_MOR_Max_woe,nombreOtorgante_8_INT_053_MOR_Max_woe,nombreOtorgante_8_INT_064_MOR_Max_woe,nombreOtorgante_8_INT_065_MOR_Max_woe,nombreOtorgante_8_INT_071_MOR_Max_woe,nombreOtorgante_8_INT_072_MOR_Max_woe,nombreOtorgante_8_INT_076_MOR_Max_woe,nombreOtorgante_8_INT_077_MOR_Max_woe,nombreOtorgante_8_INT_083_MOR_Max_woe,nombreOtorgante_8_INT_089_MOR_Max_woe
1,0.283191,-0.78465,-0.328648,-0.93795,0.110101,-0.314646,-0.681433,-0.506041,-0.494305,-0.393540,...,-0.483723,-0.478071,-0.694359,-0.358449,-0.451612,-0.389045,-0.411038,-0.460498,-0.43935,-0.420007
2,-0.384485,0.06379,0.206650,-0.93795,-0.358076,0.179455,0.223721,0.177469,0.174851,0.186906,...,0.265705,0.237728,0.223721,0.179650,0.192475,0.197558,0.202075,0.186129,0.18005,0.186383


In [28]:
variables_del_modelo = list(df_train.iloc[:,4:-1].columns)

In [29]:
X_train = df_train.iloc[:,4:-1].values ## Checar que si sean tus variables
y_train = df_train['BGI'].values


In [30]:
#df_train.iloc[:,3:]
#df_1.head(1)
df_1.iloc[:,4:].head(3)

,BGI,DOM_009_BUR_Max_woe,nombreOtorgante_1_EXP_003_Sum_woe,NUM_002_BUR_Mean_woe,RES_033_BUR_woe,INT_050_MOR_Mean_woe,nombreOtorgante_8_INT_061_MOR_Sum_woe,nombreOtorgante_8_INT_064_MOR_Sum_woe,nombreOtorgante_8_INT_066_MOR_Sum_woe,nombreOtorgante_8_INT_061_MOR_Mean_woe,...,nombreOtorgante_8_INT_052_MOR_Max_woe,nombreOtorgante_8_INT_053_MOR_Max_woe,nombreOtorgante_8_INT_064_MOR_Max_woe,nombreOtorgante_8_INT_065_MOR_Max_woe,nombreOtorgante_8_INT_071_MOR_Max_woe,nombreOtorgante_8_INT_072_MOR_Max_woe,nombreOtorgante_8_INT_076_MOR_Max_woe,nombreOtorgante_8_INT_077_MOR_Max_woe,nombreOtorgante_8_INT_083_MOR_Max_woe,nombreOtorgante_8_INT_089_MOR_Max_woe
0,0,-0.384485,0.06379,-0.328648,0.033898,-0.358076,0.170703,0.170703,0.170703,0.170703,...,0.126249,0.127186,0.170703,0.170703,0.170703,0.170703,0.170703,0.170703,0.170703,0.170703
1,0,0.283191,-0.78465,-0.328648,-0.937950,0.110101,-0.314646,-0.681433,-0.506041,-0.494305,...,-0.483723,-0.478071,-0.694359,-0.358449,-0.451612,-0.389045,-0.411038,-0.460498,-0.439350,-0.420007
2,1,-0.384485,0.06379,0.206650,-0.937950,-0.358076,0.179455,0.223721,0.177469,0.174851,...,0.265705,0.237728,0.223721,0.179650,0.192475,0.197558,0.202075,0.186129,0.180050,0.186383


In [31]:
X = df_1.iloc[:,5:].values

In [32]:
df_1["Partición"] = np.where(df_1["is_train"]==True,"1_Entrenamiento","2_Comprobación")
df_1.drop(columns="is_train",inplace = True)

In [33]:
df["Partición"] = np.where(df["is_train"]==True,"1_Entrenamiento","2_Comprobación")
df.drop(columns="is_train",inplace = True)

In [34]:
#df_1.head()
#df.head()
print(df_1.shape)
print(df.shape)



(1376, 40)
(2130, 40)


### SVM

In [45]:
from sklearn.svm import SVC

param_grid = {'C': [0.1,10,15], 'gamma': [0.10,.01,.001]}#,10,.1,
svm = GridSearchCV(SVC(probability=True, kernel = 'rbf',random_state=0),param_grid,scoring='accuracy',cv = 10, n_jobs = -1)
svm.fit(X_train,y_train)
print(svm.best_score_)
print(svm.best_estimator_)

0.7283256022744276
SVC(C=15, gamma=0.1, probability=True, random_state=0)


In [46]:
predictions = svm.predict_proba(X) #### Solo se califica la base sin smote X
predictions = np.array([ predictions[i][0] for i in range(len(predictions))])
df_1['score1'] = np.around(predictions*1000, decimals = 0)
df_1['BGI_CONS'] = df_1['BGI']
output_path = 'BASE CALIFICADA_SVM_S3.csv'
df_1.to_csv(output_path, index = False)

In [47]:
#df_tablas = pd.read_csv(output_path)
N = 5
t_train, t_test = tablas_de_entrenamiento_y_validacion(df_1, N, string_score = 'score1', string_bgi = 'BGI_CONS')

In [48]:
t_train

,SCORE,TOTAL,%ACUM,BUENOS,%BUENOS,%ACUMBUENOS,MALOS,%MALOS,%ACUMMALOS,TM ACUM,KS,ODDS,GINI
0,845-999,187,100.0%,178,95.2%,100.0%,9,4.8%,100.0%,4.8%,0.0%,19.8,38.3%
1,718-843,204,80.5%,189,92.6%,75.8%,15,7.4%,95.9%,6.1%,20.2%,12.6,39.5%
2,474-717,187,59.1%,155,82.9%,50.1%,32,17.1%,89.2%,9.7%,39.1%,4.8,32.0%
3,299-472,186,39.6%,130,69.9%,29.0%,56,30.1%,74.8%,14.7%,45.8%,2.3,24.2%
4,55-297,193,20.2%,83,43.0%,11.3%,110,57.0%,49.5%,23.2%,38.3%,0.8,10.0%
5,TOTAL,957,,735,,,222,23.2%,,,45.8%,,43.9%


In [49]:
t_test

,SCORE,TOTAL,%ACUM,BUENOS,%BUENOS,%ACUMBUENOS,MALOS,%MALOS,%ACUMMALOS,TM ACUM,KS,ODDS,GINI
0,849-998,88,100.0%,81,92.0%,100.0%,7,8.0%,100.0%,8.0%,0.0%,11.6,40.4%
1,719-843,71,79.0%,62,87.3%,75.5%,9,12.7%,92.1%,10.1%,16.7%,6.9,29.5%
2,481-711,83,62.1%,64,77.1%,56.7%,19,22.9%,82.0%,14.5%,25.4%,3.4,28.3%
3,299-472,83,42.2%,59,71.1%,37.3%,24,28.9%,60.7%,18.2%,23.4%,2.5,18.7%
4,42-297,94,22.4%,64,68.1%,19.4%,30,31.9%,33.7%,21.2%,14.3%,2.1,7.6%
5,TOTAL,419,,330,,,89,21.2%,,,25.4%,,24.4%


In [ ]:
tables_to_excel([t_train,t_test], ['Train','Test'], 'Tablas de validación SVM_S2')

In [ ]:
joblib.dump(svm, 'SVM.pkl')
 
# some time later...
 
# load the model from disk
loaded_model = joblib.load('SVM.pkl')

### Regresión Logística

In [ ]:
from sklearn.linear_model import LogisticRegression

param_grid = {'C': [0.1,1,15,20]}
logreg_cv = GridSearchCV(LogisticRegression(solver = 'liblinear',random_state = 12),param_grid,scoring='accuracy',cv = 10, n_jobs = -1)
logreg_cv.fit(X_train, y_train) #logreg_cv.fit(X_train, y_train)


In [ ]:
score = logreg_cv.score(df_1[df_1['Partición'] == '2_Comprobación'][variables_del_modelo], df_1[df_1['Partición'] == '2_Comprobación']['BGI'])
print('Test Accuracy Score', score)

In [ ]:
predictions = logreg_cv.predict_proba(X) #### Solo se califica la base sin smote
predictions = np.array([ predictions[i][0] for i in range(len(predictions))])
df_1['score1'] = np.around(predictions*1000, decimals = 0)
df_1['BGI_CONS'] = df_1['BGI']
output_path = 'C:/Users/JavierJhairtLópezRoj/OneDrive - Exitus Credit/Escritorio/SEP ACTIVOS 2/Tablas de validacion, bases calificadas, modelos aplicados/Bases calificadas/S1/BASE CALIFICADA_S1_RL.csv'

#df_1.to_csv(output_path, index = False)

In [ ]:
#df_tablas = pd.read_csv(output_path)
t_train, t_test = tablas_de_entrenamiento_y_validacion(df_1, N, string_score = 'score1', string_bgi = 'BGI_CONS')

In [ ]:
t_train

In [ ]:
t_test

In [ ]:
tables_to_excel([t_train,t_test], ['Train','Test'], 'Tablas de validación RL S2')

In [ ]:
joblib.dump(logreg_cv, 'RL_S2.pkl')
 
# some time later...
 
# load the model from disk
loaded_model = joblib.load('RL_S2.pkl')

### MLP

In [70]:
from sklearn.neural_network import MLPClassifier
ann = MLPClassifier(alpha=1, hidden_layer_sizes=(2,8,8,4,2), activation = 'identity',
                    solver = 'adam', learning_rate = 'adaptive', learning_rate_init = 0.1,random_state=12)
ann.fit(X_train,y_train)

MLPClassifier(activation='identity', alpha=1,
              hidden_layer_sizes=(2, 8, 8, 4, 2), learning_rate='adaptive',
              learning_rate_init=0.1, random_state=12)

In [71]:
predictions = ann.predict_proba(X) #### Solo se califica la base sin smote
predictions = np.array([ predictions[i][0] for i in range(len(predictions))])
df_1['score1'] = np.around(predictions*1000, decimals = 0)
#predictions = ann.predict_proba(X) #### Solo se califica la base sin smote
df_1['BGI_CONS'] = df_1['BGI']
output_path = 'BASE CALIFICADA_MLP_S2.csv'

df_1.to_csv(output_path, index = False)

In [72]:
#df_tablas = pd.read_csv(output_path)
t_train, t_test = tablas_de_entrenamiento_y_validacion(df_1, N, string_score = 'score1', string_bgi = 'BGI_CONS')

In [73]:
t_train

,SCORE,TOTAL,%ACUM,BUENOS,%BUENOS,%ACUMBUENOS,MALOS,%MALOS,%ACUMMALOS,TM ACUM,KS,ODDS,GINI
0,585-806,179,100.0%,167,93.3%,100.0%,12,6.7%,100.0%,6.7%,0.0%,13.9,36.4%
1,489-584,197,81.3%,169,85.8%,77.3%,28,14.2%,94.6%,10.6%,17.3%,6.0,36.3%
2,435-487,165,60.7%,136,82.4%,54.3%,29,17.6%,82.0%,12.8%,27.7%,4.7,26.0%
3,341-434,200,43.5%,141,70.5%,35.8%,59,29.5%,68.9%,17.3%,33.1%,2.4,23.3%
4,119-339,216,22.6%,122,56.5%,16.6%,94,43.5%,42.3%,23.2%,25.7%,1.3,9.6%
5,TOTAL,957,,735,,,222,23.2%,,,33.1%,,31.6%


In [74]:
t_test

,SCORE,TOTAL,%ACUM,BUENOS,%BUENOS,%ACUMBUENOS,MALOS,%MALOS,%ACUMMALOS,TM ACUM,KS,ODDS,GINI
0,589-806,85,100.0%,77,90.6%,100.0%,8,9.4%,100.0%,9.4%,0.0%,9.6,38.7%
1,492-584,89,79.7%,76,85.4%,76.7%,13,14.6%,91.0%,12.1%,14.3%,5.8,35.6%
2,436-488,71,58.5%,59,83.1%,53.6%,12,16.9%,76.4%,13.5%,22.8%,4.9,23.6%
3,342-434,89,41.5%,70,78.7%,35.8%,19,21.3%,62.9%,15.6%,27.2%,3.7,22.2%
4,86-339,85,20.3%,48,56.5%,14.5%,37,43.5%,41.6%,21.2%,27.0%,1.3,8.4%
5,TOTAL,419,,330,,,89,21.2%,,,27.2%,,28.5%


In [75]:
tables_to_excel([t_train,t_test], ['Train','Test'], 'Tablas de validación MLP_S3')

In [77]:
joblib.dump(ann, 'MLP_S3.pkl')
 
# some time later...
 
# load the model from disk
loaded_model = joblib.load('MLP_S3.pkl')

### Red Nueronal Recurrente

In [401]:
#define the keras model
from keras.constraints import max_norm
model = Sequential([
    Dense(8, activation='relu',input_dim= X_train.shape[1]),
    #Dense(10, activation= 'relu', kernel_regularizer= L1(0.1)),
    #model.add(Dropout(0.3))
    #Dropout(0.7),
    #Dense(8, activation='relu', kernel_regularizer= L2(0.1)),
    #Dropout(0.1),
    Dense(100, activation='relu'),#, kernel_regularizer= L2(0.1)
    Dropout(0.6),
    
    #Dense(100, activation='relu', kernel_regularizer= L2(0.3)),
        
    # Dense(80, activation='relu', kernel_regularizer= L2(0.2)),#, kernel_regularizer= L2()),
    # Dropout(0.3),
    #model.add(Dropout(0.3))

    Dense(40, activation='relu'),#, kernel_regularizer= L2()),
    Dropout(0.5),

    
    Dense(4, activation='relu'),#, kernel_regularizer= L2()),
    #model.add(Dropout(0.3))
    Dense(2, activation='softmax')
]
)
 
# compile the keras model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 
# fit the keras model on the dataset
history = model.fit(X_train, y_train, epochs= 12,
                    validation_data=(df_1[df_1['Partición'] == '2_Comprobación'][variables_del_modelo], df_1[df_1['Partición'] == '2_Comprobación']['BGI']),
                    verbose = 2)

Epoch 1/12
52/52 - 1s - loss: 0.6887 - accuracy: 0.5524 - val_loss: 0.6904 - val_accuracy: 0.7232
Epoch 2/12
52/52 - 0s - loss: 0.6792 - accuracy: 0.5805 - val_loss: 0.7094 - val_accuracy: 0.6754
Epoch 3/12
52/52 - 0s - loss: 0.6750 - accuracy: 0.6130 - val_loss: 0.6768 - val_accuracy: 0.7088
Epoch 4/12
52/52 - 0s - loss: 0.6503 - accuracy: 0.6350 - val_loss: 0.6561 - val_accuracy: 0.6492
Epoch 5/12
52/52 - 0s - loss: 0.6297 - accuracy: 0.6620 - val_loss: 0.6760 - val_accuracy: 0.5800
Epoch 6/12
52/52 - 0s - loss: 0.6126 - accuracy: 0.6920 - val_loss: 0.6983 - val_accuracy: 0.5847
Epoch 7/12
52/52 - 0s - loss: 0.6071 - accuracy: 0.6859 - val_loss: 0.6511 - val_accuracy: 0.6253
Epoch 8/12
52/52 - 0s - loss: 0.6067 - accuracy: 0.6871 - val_loss: 0.6958 - val_accuracy: 0.5895
Epoch 9/12
52/52 - 0s - loss: 0.5875 - accuracy: 0.7024 - val_loss: 0.6447 - val_accuracy: 0.6348
Epoch 10/12
52/52 - 0s - loss: 0.5879 - accuracy: 0.7024 - val_loss: 0.7004 - val_accuracy: 0.6062
Epoch 11/12
52/52 -

In [402]:
output_model_path = 'RNR_S3_p1.h5'#onde #M es el numero del modelo.

#guardar la imagen de cada modelo
model.save(output_model_path)

In [403]:
#Para hacer predicciones sobre nuevos modelos
from keras.models import load_model

model = load_model(output_model_path)

In [404]:
predictions = model.predict(X)
df_1['score1'] = np.around(predictions[:,0]*1000, decimals = 0)
df_1['BGI_CONS'] = df_1['BGI']
df_1.head()

,idconsulta,IND_2,aceptado,BGI,DOM_009_BUR_Max_woe,nombreOtorgante_1_EXP_003_Sum_woe,NUM_002_BUR_Mean_woe,RES_033_BUR_woe,INT_050_MOR_Mean_woe,nombreOtorgante_8_INT_061_MOR_Sum_woe,...,nombreOtorgante_8_INT_071_MOR_Max_woe,nombreOtorgante_8_INT_072_MOR_Max_woe,nombreOtorgante_8_INT_076_MOR_Max_woe,nombreOtorgante_8_INT_077_MOR_Max_woe,nombreOtorgante_8_INT_083_MOR_Max_woe,nombreOtorgante_8_INT_089_MOR_Max_woe,Partición,score1,BGI_CONS,score1_TILN
0,942380,2021_2,0,0,-0.384485,0.063790,-0.328648,0.033898,-0.358076,0.170703,...,0.170703,0.170703,0.170703,0.170703,0.170703,0.170703,2_Comprobación,240.0,0,1
1,857713,2020_11,0,0,0.283191,-0.784650,-0.328648,-0.937950,0.110101,-0.314646,...,-0.451612,-0.389045,-0.411038,-0.460498,-0.439350,-0.420007,1_Entrenamiento,442.0,0,1
2,911339,2021_1,0,1,-0.384485,0.063790,0.206650,-0.937950,-0.358076,0.179455,...,0.192475,0.197558,0.202075,0.186129,0.180050,0.186383,1_Entrenamiento,263.0,1,1
3,801699,2020_9,0,1,-0.384485,0.230081,0.271255,-0.937950,-0.358076,-0.731443,...,-0.818092,-0.878717,-0.704111,-0.861190,-0.882630,-0.905515,1_Entrenamiento,227.0,1,1
4,903992,2021_1,0,0,0.729158,-0.784650,-0.328648,0.033898,0.783966,0.179455,...,0.192475,0.197558,0.202075,0.186129,0.180050,0.186383,1_Entrenamiento,726.0,0,3


In [405]:
df = df_1

### Tabla de Validación

##### ENTRENAMIENTO

In [406]:
output_path = 'BASE CALIFICADA_RNR_p1_S3.csv'

In [407]:
df_1.to_csv(output_path)

In [408]:
#df_tablas = pd.read_csv(output_path)
N = 5
t_train, t_test = tablas_de_entrenamiento_y_validacion(df,N, string_score = 'score1', string_bgi = 'BGI')


In [411]:
t_train

,SCORE,TOTAL,%ACUM,BUENOS,%BUENOS,%ACUMBUENOS,MALOS,%MALOS,%ACUMMALOS,TM ACUM,KS,ODDS,GINI
0,741-963,185,100.0%,171,92.4%,100.0%,14,7.6%,100.0%,7.6%,0.0%,12.2,37.4%
1,615-735,182,80.7%,163,89.6%,76.7%,19,10.4%,93.7%,9.0%,17.0%,8.6,34.0%
2,415-614,210,61.7%,168,80.0%,54.6%,42,20.0%,85.1%,13.0%,30.6%,4.0,33.2%
3,268-414,193,39.7%,137,71.0%,31.7%,56,29.0%,66.2%,17.0%,34.5%,2.4,21.6%
4,194-266,187,19.5%,96,51.3%,13.1%,91,48.7%,41.0%,23.2%,27.9%,1.1,8.0%
5,TOTAL,957,,735,,,222,23.2%,,,34.5%,,34.3%


In [412]:
t_test

,SCORE,TOTAL,%ACUM,BUENOS,%BUENOS,%ACUMBUENOS,MALOS,%MALOS,%ACUMMALOS,TM ACUM,KS,ODDS,GINI
0,737-962,87,100.0%,79,90.8%,100.0%,8,9.2%,100.0%,9.2%,0.0%,9.9,39.7%
1,618-735,78,79.2%,66,84.6%,76.1%,12,15.4%,91.0%,12.1%,15.0%,5.5,31.4%
2,418-614,80,60.6%,66,82.5%,56.1%,14,17.5%,77.5%,13.9%,21.5%,4.7,26.6%
3,267-414,85,41.5%,62,72.9%,36.1%,23,27.1%,61.8%,17.3%,25.7%,2.7,19.8%
4,194-263,89,21.2%,57,64.0%,17.3%,32,36.0%,36.0%,21.2%,18.7%,1.8,7.6%
5,TOTAL,419,,330,,,89,21.2%,,,25.7%,,25.1%


In [413]:
tables_to_excel([t_train,t_test],['Train','Test'],'Tablas de validación RNR_p1_S3')